<a href="https://colab.research.google.com/github/galb4tosha/my-ml/blob/master/Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras import utils
import tensorflow as tf
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import numpy.fft
import numpy as np
import statistics
from scipy.io import wavfile
import scipy.io.wavfile as wav
import numpy as np

import scipy.io.wavfile as wav


from sklearn import tree, linear_model, ensemble
from sklearn.metrics import accuracy_score
%matplotlib inline 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
list_files_jackson_test = ["{}_jackson_{}.wav".format(i,j) for i in range(10) for j in range(5)]
list_files_nicolas_test = ["{}_nicolas_{}.wav".format(i,j) for i in range(10) for j in range(5)]
list_files_theo_test = ["{}_theo_{}.wav".format(i,j) for i in range(10) for j in range(5)]
list_files_yweweler_test = ["{}_yweweler_{}.wav".format(i,j) for i in range(10) for j in range(5)]

list_files_jackson_train = ["{}_jackson_{}.wav".format(i,j) for i in range(10) for j in range(5,50)]
list_files_nicolas_train = ["{}_nicolas_{}.wav".format(i,j) for i in range(10) for j in range(5,50)]
list_files_theo_train = ["{}_theo_{}.wav".format(i,j) for i in range(10) for j in range(5,50)]
list_files_yweweler_train = ["{}_yweweler_{}.wav".format(i,j) for i in range(10) for j in range(5,50)]

In [0]:
path = "/content/drive/My Drive/records/"
files_dir_train = np.array([])
files_dir_train = np.append(list_files_jackson_train,[list_files_nicolas_train,list_files_theo_train,list_files_yweweler_train])
files_dir_test = np.array([])
files_dir_test = np.append(list_files_jackson_test,[list_files_nicolas_test,list_files_theo_test,list_files_yweweler_test])

**Обработка данных**

In [0]:
!pip install python_speech_features

In [0]:
from python_speech_features import mfcc

In [0]:
def create_mfcc(files): # Генерация mfcc из wav файла
  sounds = np.array([wav.read(path + f) for f in files])
  X = np.array([mfcc(s[1],s[0]) for s in sounds])
  return X

In [0]:
X_train = create_mfcc(files_dir_train)
X_test = create_mfcc(files_dir_test)

Для обучения необходимо чтобы данные были одного размера. Я приводил к медианному значению длины всех массивов, так как в выборке есть несколько файлов, которые имеют очень большую длину. Также необходимо, чтобы X_train и X_test имел вид массива массивов, а не массив матриц, которые возвращает функция mfcc.

In [0]:
x_len = int(statistics.median(np.array([len(x) for x in X_train]))) 

In [0]:
def formating_size_X(X,size):
  X_buf = np.copy(X)
  for i in range(len(X)):
    if len(X[i]) > size:
      X_buf[i] = X[i][:size]
    else:
      adding_z_len = size - len(X[i])
      buf_vect = np.array([[0 for j in range(13)] for i in range(adding_z_len)])
      X_buf[i] = np.append(X[i],buf_vect)
  return X_buf

In [0]:
X_train = formating_size_X(X_train,x_len)
X_test = formating_size_X(X_test,x_len)

In [0]:
train_X = np.array([np.reshape(i,13*x_len) for i in X_train])
X_train = np.copy(train_X)
test_X = np.array([np.reshape(i,13*x_len) for i in X_test])
X_test = np.copy(test_X)

Формирование матриц Y было сделано с помощью первого символа названия файла. Также для обучения нейросети Y_train должен иметь категориальный вид.


In [0]:
Y_train = np.array([int(f[0]) for f in files_dir_train])
Y_test = np.array([int(f[0]) for f in files_dir_test])
Y_train_net = utils.to_categorical(Y_train,10)
Y_test_net = utils.to_categorical(Y_test,10)

**Decision Tree**

In [0]:
model_tree = tree.DecisionTreeClassifier(min_samples_leaf = 3, max_depth = 10, random_state = 1)
model_tree.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best')

In [0]:
predictions = model_tree.predict(X_train)
accuracy_score(Y_train,predictions)

0.9288888888888889

In [0]:
predictions_test = model_tree.predict(X_test)
accuracy_score(Y_test,predictions_test)

0.74

**Logistic Regression**

In [0]:
model_log = linear_model.LogisticRegression(solver = "lbfgs", multi_class = "multinomial")
model_log.fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predictions_log = model_log.predict(X_train)
accuracy_score(Y_train,predictions_log)

1.0

In [0]:
predictions_log_test = model_log.predict(X_test)
accuracy_score(Y_test,predictions_log_test)

0.975

**Random forest**

In [0]:
model_forest = ensemble.RandomForestClassifier(min_samples_leaf = 1,max_depth = 50,n_estimators=140, random_state = 1)
model_forest.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=50, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=140,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [0]:
predictions_forest = model_forest.predict(X_train)
accuracy_score(Y_train,predictions_forest)

1.0

In [0]:
predictions_forest_test = model_forest.predict(X_test)
accuracy_score(Y_test,predictions_forest_test)

0.99

**Gradient Boosting**

In [0]:
model_boost = ensemble.GradientBoostingClassifier(min_samples_leaf = 3,max_depth = 120, n_estimators = 100, random_state = 1)
model_boost.fit(X_train,Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=120,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=3, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
predictions_boost = model_boost.predict(X_train)
accuracy_score(Y_train,predictions_boost)

1.0

In [0]:
model_boost_test = model_boost.predict(X_test)
accuracy_score(Y_test,model_boost_test)

0.91

**Tensorflow**

In [0]:
model = Sequential()

model.add(Dense(800, input_dim=13*x_len, activation="relu"))
model.add(Dense(400, input_dim=200, activation="relu"))
model.add(Dense(200, input_dim=100, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(100, input_dim=50, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(10, activation = "softmax"))

In [0]:
model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

print(model.summary())

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_115 (Dense)            (None, 800)               396000    
_________________________________________________________________
dense_116 (Dense)            (None, 400)               320400    
_________________________________________________________________
dense_117 (Dense)            (None, 200)               80200     
_________________________________________________________________
dropout_39 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_118 (Dense)            (None, 100)               20100     
_________________________________________________________________
dropout_40 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_119 (Dense)            (None, 10)              

In [0]:
model.fit(X_train, Y_train_net, 
          batch_size=100, 
          epochs=24,
          validation_split=0.01,
          verbose=1)

Train on 1782 samples, validate on 18 samples
Epoch 1/24
1782/1782 [==============================] - 1s 412us/sample - loss: 3.6834 - acc: 0.4119 - val_loss: 0.8573 - val_acc: 0.6111
Epoch 2/24
1782/1782 [==============================] - 0s 56us/sample - loss: 0.7577 - acc: 0.7430 - val_loss: 0.5292 - val_acc: 0.7778
Epoch 3/24
1782/1782 [==============================] - 0s 53us/sample - loss: 0.4630 - acc: 0.8395 - val_loss: 0.1342 - val_acc: 0.9444
Epoch 4/24
1782/1782 [==============================] - 0s 56us/sample - loss: 0.3684 - acc: 0.8799 - val_loss: 0.1972 - val_acc: 0.9444
Epoch 5/24
1782/1782 [==============================] - 0s 54us/sample - loss: 0.2951 - acc: 0.9057 - val_loss: 0.0618 - val_acc: 0.9444
Epoch 6/24
1782/1782 [==============================] - 0s 54us/sample - loss: 0.2327 - acc: 0.9231 - val_loss: 0.0789 - val_acc: 0.9444
Epoch 7/24
1782/1782 [==============================] - 0s 56us/sample - loss: 0.1721 - acc: 0.9428 - val_loss: 0.0883 - val_acc: 1

In [0]:
model.evaluate(X_test,Y_test_net,verbose = 1)

200/200 [==============================] - 0s 142us/sample - loss: 0.0628 - acc: 0.9800


[0.06277705874294043, 0.98]

In [0]:
a = np.array([np.random.normal() for i in range(10)])

In [0]:
for i, x in enumerate(a):
  print(f'{i}:{x}')

0:-0.8477918217587281
1:0.6290351578205852
2:-0.012270458753462912
3:1.2770388577297536
4:1.3848277068509496
5:-0.9960808815651125
6:0.8448780402612608
7:-0.3941531318690586
8:-0.7275935458080751
9:0.14202880014032837
